In [13]:
import toyplot
import numpy as np

In [14]:
## make a canvas
canvas = toyplot.Canvas(width=500, height=350)

## vertex coordinates
xcoords = [0, 2, 3, 4]
ycoords = [0, 3, 1, 1]

## as arrays
coords = np.array([xcoords, ycoords]).T
edges = np.array([[0, 1], [0, 2], [1, 2], [2, 3]])

## plot graph 
axes = canvas.cartesian(bounds=(50, 450, 50, 250))
axes.graph(edges, vcoordinates=coords, ewidth=3, vsize=20)
axes.x.ticks.labels.show = False

## plot scatter
axes = canvas.cartesian(bounds=(50, 450, 275, 300))
axes.scatterplot(xcoords, [100 for i in coords], size=15)
axes.show = False

## plot text
axes = canvas.cartesian(bounds=(50, 450, 300, 350))
axes.text(xcoords, [100 for i in coords],
          text=range(4), 
          color=toyplot.color.Palette()[1],
          style={'font-size': '20px', 
                 "text-anchor":"start",}
          )
axes.show = False

0 1 2 3 0 1 2 3 0 1 2 3 
 
 
 Save as .csv

In [149]:
class Tree(object):
    def __init__(self, newick=None, admix=None):

        ## use default newick string if not given
        if newick:
            self.newick = newick
            self.admix = admix
        else:
            self.newick = "((((a,b),c),d), ((((e,f),g),h) , (((i,j),k),l)));"
        ## parse newick, assigns idx to nodes, returns tre, edges, verts, names
        t, e, v, names = cladogram(self.newick)

        ## parse admixture events
        self.admix = admix
        self._check_admix()

        ## store values
        self.tree = t
        self.edges = e
        self.verts = v
        self.names = names.values()  ## in tree plot vlshow order


    def _check_admix(self):
        ## raise an error if admixture event is not possible in time period
        if self.admix:
            for event in self.admix:
                pass #print(event)


    def draw(self, yaxis=False, show_tips=False, use_edge_lengths=True, **kwargs):
        """
        plot the tree using toyplot.graph. 

        Parameters:
        -----------
            ... 
            yaxis: bool
                Show the y-axis.
            use_edge_lengths: bool
                Use edge lengths from newick tree.
            ... 

        """
        ## have to re-make the vert if not using edges
        if use_edge_lengths:
            verts = self.verts
        else:
            _, _, v, _ = cladogram(self.newick, use_edge_lengths=False)
            verts = v

        ## update kwargs
        args = {"height": min(1000, 15*len(self.tree)),
                "width": min(1000, 15*len(self.tree)),
                "vsize": 20, 
                "vlshow": True, 
                "ewidth": 3, 
                "vlstyle": {"font-size": "18px"}}
        args.update(kwargs)

        ## create a canvas and a single cartesian coord system
        canvas = toyplot.Canvas(height=args['height'], width=args['width'])
        axes = canvas.cartesian(bounds=("5%", "95%", "5%", "95%"))
            
        ## get units of height, 
        if use_edge_lengths == False:
            unit = 1.
        else:
            unit = verts[:, 1].max() / 10.
            
        ## add +1 unit for each data col to be added
        ## ....
        
        ## add +X for name space
        #treebot = verts[:, 1] + unit*5
        #verts[:, 1] = treebot[:, 1].min()

        ## hide x and hide/show y axies
        axes.x.show = False
        if yaxis:
            axes.y.show = True
            #axes.y.spine.position = treebot
        else:
            axes.y.show = False        

        ## add the graph
        mark1 = axes.graph(self.edges, 
                          vcoordinates=verts, 
                          ewidth=args["ewidth"], 
                          ecolor=toyplot.color.near_black, 
                          vlshow=args["vlshow"],
                          vsize=args["vsize"],
                          vlstyle=args["vlstyle"],
                          vlabel=self.names)   

        ## get name locations
        nams = [i for i in self.names if not isinstance(i, int)]
        locs = verts[verts[:, 1] == 0]        

        ## add scatterplot data
        ys = np.array([-0.5*unit for i in range(locs.shape[0])])
        mark2 = axes.scatterplot(locs[:, 0], ys, size=20)
        
        ## add names to tips
        if show_tips:
            ys = np.array([-0.5*unit for i in range(locs.shape[0])])
            mark3 = axes.text(locs[:, 0], ys, nams,
                              angle=-90, 
                              color=toyplot.color.near_black,
                              style={
                                 "font-size": "18px", 
                                 "-toyplot-anchor-shift":"0",
                                 #"text-anchor":"start",
                                 },
                              ) 

        ## plot admix lines ---------------------------------
        if self.admix:
            for event in self.admix:
                ## get event
                source, sink, _, _, _ = event

                ## get nodes from tree
                source = self.tree.search_nodes(name=source)[0]
                sink = self.tree.search_nodes(name=sink)[0]

                ## get coordinates
                fromx = np.max([source.up.x, source.x]) - np.abs(source.up.x - source.x) / 2.
                fromy = source.y + (source.up.y - source.y) / 2.
                tox = np.max([sink.up.x, sink.x]) - np.abs(sink.up.x - sink.x) / 2.
                toy = sink.y + (sink.up.y - sink.y) / 2.

                mark = axes.plot([fromx, tox], [fromy, toy], 
                                color=toyplot.color.Palette()[1], 
                                style={"stroke-width": 3, 
                                       "stroke-dasharray": "2, 2"},
                                )

        ## return plotting 
        return canvas, axes

In [150]:
#from ipyrad.analysis.baba import *
x = Tree(admix=[['i', 'k', 0, 1, 0.005]])

In [151]:
x.draw(width=400, height=300, yaxis=True, show_tips=True, 
       vlshow=False, vsize=0);

a b c d e f g h i j k l 0 2 5 
 
 
 Save as .csv